DATA643- Project 2
======
Prashanth Padebetttu, Parshu Rath, Adjare Windokun
----------
<b> Summer 2016 <br>
Instructor: Andrew Catlin

<b> Load 100K movie data as pandas dataframe:

In [1]:
# Movielens data with 100,000 records

In [2]:
#Load data into a pandas dataframe
import csv
#import os
import pandas as pd
import numpy as np
from scipy import spatial
url = 'https://raw.githubusercontent.com/ppadebettu/CUNY/IS-607-Data-Acquisition-and-Management/IS643/Project_2/movielens.csv'
df = pd.read_csv(url, sep = "," , header = 0, na_values='NaN')


In [3]:
df.head()

,userid,age,gender,occupation,zipcode,itemid,rating,timestamp,movieid,movietitle,...,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,196,49,M,writer,55105,242,3,881250949,242,Kolya (1996),...,0,0,0,0,0,0,0,0,0,0
1,186,39,F,executive,00000,302,3,891717742,302,L.A. Confidential (1997),...,0,1,0,0,1,0,0,1,0,0
2,22,25,M,writer,40206,377,1,878887116,377,Heavyweights (1994),...,0,0,0,0,0,0,0,0,0,0
3,244,28,M,technician,80525,51,2,880606923,51,Legends of the Fall (1994),...,0,0,0,0,0,1,0,0,1,1
4,166,47,M,educator,55113,346,1,886397596,346,Jackie Brown (1997),...,0,0,0,0,0,0,0,0,0,0


<b>Load 100K movie data as dictionary:

In [24]:
# Get movie titles
#url = 'https://raw.githubusercontent.com/prcuny/DATA643/master/Project-2/'
movies={}
#for line in open(url+'u.item', 'r'):  #does not work yet
for line in open('u.item'):    
    (id,title)=line.split('|')[0:2]
    movies[id]=title


In [26]:
# Load user ratings
mdict={}
for line in open('u.data'):
   (user,movieid,rating,ts)=line.split('\t')
   mdict.setdefault(user,{})
   mdict[user][movies[movieid]]=float(rating)


Check to see what all the movies user '20' has rated.

In [37]:
mdict['20']

{'African Queen, The (1951)': 3.0,
 'Aladdin (1992)': 3.0,
 'Aliens (1986)': 2.0,
 'Back to the Future (1985)': 3.0,
 'Beauty and the Beast (1991)': 4.0,
 'Blues Brothers, The (1980)': 3.0,
 'Braveheart (1995)': 5.0,
 'Christmas Carol, A (1938)': 4.0,
 "Dante's Peak (1997)": 4.0,
 'Die Hard (1988)': 2.0,
 'E.T. the Extra-Terrestrial (1982)': 2.0,
 'Empire Strikes Back, The (1980)': 3.0,
 'Eraser (1996)': 3.0,
 'Forrest Gump (1994)': 1.0,
 'Ghost and the Darkness, The (1996)': 5.0,
 'Happy Gilmore (1996)': 1.0,
 'Home Alone (1990)': 2.0,
 'Independence Day (ID4) (1996)': 3.0,
 'Indiana Jones and the Last Crusade (1989)': 4.0,
 'Island of Dr. Moreau, The (1996)': 1.0,
 "It's a Wonderful Life (1946)": 5.0,
 'Jungle2Jungle (1997)': 4.0,
 'Jurassic Park (1993)': 4.0,
 'Lost World: Jurassic Park, The (1997)': 4.0,
 'Michael (1996)': 1.0,
 'Miracle on 34th Street (1994)': 3.0,
 'Mission: Impossible (1996)': 3.0,
 "Mr. Holland's Opus (1995)": 4.0,
 "One Flew Over the Cuckoo's Nest (1975)": 1.0